# N-queens problem
The problem is to place $n$ queens on an $n\times n$ chess board so that they don't attack each other. In other words, we have to *choose* where the queens have to be on the board to fulfill this objective. This directly leads to the idea that we will have variables $x_{i,j}$, which equals 1 if there is a queen on the field at row $i$ and column $j$, and 0 otherwise.

> This is a big departure from our previous exercises, where we assumed the variables to be continuous between some lower and upper bound. Suddenly, we now have the option to choose, which opens up amazing possibilities for modelling complex decisions.

In [1]:
import xpress as xp
import numpy as np
import math
from dataclasses import dataclass
from sys import stdout

model = xp.problem("n_queens")
n = 8 # The number of queens
nRange = range(n)

# Define the indices used
@dataclass(frozen=True)
class Position:
    row: int
    column: int
        
# Generate the list of all positions used
positions = [Position(i,j) for i in nRange for j in nRange]

# Enable retrieval of position based on index
def get_pos(positions: list, i: int, j: int):
    return [pos for pos in positions if pos.row == i and pos.column == j].pop()

# Define the variables
x = {pos : xp.var(vartype = xp.binary, name=f'x_{pos.row},{pos.column}') 
     for pos in positions}
model.addVariable(x)

Now that we have described the variables, we have to "encode" the way queens attack each other in chess. Specifically, a queen attacks anything that is in the same row, column or diagonal. This means, if they should not attack each other, you only can have one queen per column, row or diagonal, and this we need to encode mathematically:

## Only one queen per column
This is fairly easy. If we look over all the rows for a given column $j$, we only can have one queen:
\begin{equation}
\sum \limits_{i} x_{i,j} \leq 1, \hspace{0.15cm} \forall j
\end{equation}

In [2]:
one_queen_per_column = (xp.constraint(xp.Sum(x[pos] 
                                             for pos in positions 
                                             if pos.column == j) <= 1, 
                                      name = f'One queen for column {j}')
                       for j in nRange)

## Only one queen per row
The same logic as for "One queen per column" applies, just with swapped indices:
\begin{equation}
\sum \limits_{j} x_{i,j} \leq 1, \hspace{0.15cm} \forall i
\end{equation}

In [4]:
one_queen_per_row = (xp.constraint(xp.Sum(x[pos] for pos in positions if pos.row == i) <= 1, name = f'One queen for row {i}')
                       for i in nRange)

## Only one queen per diagonal
This is the tricky one, of course. First off, we identify that we have to diagonals: bottom right to top left and bottom left to top right. Let's consider the simplest case, the main diagonal from $(0,0)$ to $(n,n)$. Then, we get:
\begin{align}
\sum \limits_{k=0}^{n} x_{k,k} \leq 1
\end{align}
Moving "one to the right", we get:
\begin{align}
\sum \limits_{k=0}^{n-1} x_{1+k,k} \leq 1
\end{align}
Thus, generalizing we get
\begin{align}
\sum \limits_{k=0}^{n-i} x_{i+k,k} \leq 1
\end{align}
Now moving "one downwards", we get:
\begin{align}
\sum \limits_{k=0}^{n-1} x_{k,k+1} \leq 1
\end{align}
and generalizing this means:
\begin{align}
\sum \limits_{k=0}^{n-i} x_{k,k+i} \leq 1
\end{align}

Now for the diagonal from bottom left to top right, we get:
\begin{align}
\sum \limits_{k=0}^{i+1} x_{k,i-k} \leq 1, \hspace{0.3cm} \forall i \in [1,n]
\end{align}
while for the bottom side we get:
\begin{align}
\sum \limits_{k=0}^{i+1} x_{n-k-1,n+k-i-1} \leq 1, \hspace{0.3cm} \forall i \in [1,n]
\end{align}

Note: there are a lot of correct ways to describe this constraint, this is just one of them!

In [5]:
one_queen_per_first_diagonal_rows = (xp.constraint(xp.Sum(x[get_pos(positions, k+i, k)] for k in range(n - i)) <= 1, 
                                              name=f'First diagonal, first part for row {i}') for i in nRange)
one_queen_per_first_diagonal_columns = (xp.constraint(xp.Sum(x[get_pos(positions, k, k+i)] for k in range(n - i)) <= 1, 
                                              name=f'First diagonal, second part for row {i}') for i in nRange)
one_queen_per_second_diagonal_columns = (xp.constraint(xp.Sum(x[get_pos(positions, k, i-k)] for k in range(i+1)) <= 1, 
                                              name=f'Second diagonal, first part for row {i}') for i in nRange)
one_queen_per_second_diagonal_rows = (xp.constraint(xp.Sum(x[get_pos(positions, n-k-1, n+k-i-1)] for k in range(i+1)) <= 1, 
                                              name=f'Second diagonal, second part for row {i}') for i in nRange)

## Enforce $n$ queens on the board:
Finally, if we don't enforce that we have to have $n$ queens on the board, the optimizer will simply not put any. So we need to do that:
\begin{equation}
\sum \limits_{i,j} x_{i,j} = n
\end{equation}

In [7]:
n_queens_on_board = xp.constraint(xp.Sum(x[pos] for pos in positions) == n, 
                                  name=f'Ensure {n} queens on board')

## Add constraints, solve the model and post-process

In [8]:
model.addConstraint(one_queen_per_column, one_queen_per_row, one_queen_per_first_diagonal_rows, one_queen_per_first_diagonal_columns, 
                    one_queen_per_second_diagonal_columns, one_queen_per_second_diagonal_rows, n_queens_on_board)

model.solve()

In [9]:
xVal = {pos : model.getSolution(x[pos]) for pos in positions}
stdout.write("Solution:\n")
for i in nRange:
    if (i > 0):
        stdout.write('\n')
    for j in nRange:
        stdout.write(" ")
        stdout.write("Q" if xVal[get_pos(positions,i,j)] >0.5 else ".")
stdout.write("\n")

Solution:
 . Q . . . . . .
 . . . . Q . . .
 . . . . . . Q .
 . . . Q . . . .
 Q . . . . . . .
 . . . . . . . Q
 . . . . . Q . .
 . . Q . . . . .


In [ ]:
model.getPro